In [84]:
import jax
import jax.numpy as jnp
import matplotlib.pyplot as plt
from ipywidgets import interact
from ml_collections import ConfigDict
from models.ETD_KT_CM_JAX_Vectorised import *
from filters import resamplers
from filters.filter import ParticleFilter
jax.config.update("jax_enable_x64", True)


Initialisation

In [85]:
signal_params = ConfigDict(KS_params)
print(signal_params)

Advection_basis_name: none
E: 1
Forcing_basis_name: none
P: 0
S: 0
c_0: 0
c_1: 1
c_2: 1
c_3: 0.0
c_4: 1
dt: 0.25
equation_name: Kuramoto-Sivashinsky
initial_condition: Kassam_Trefethen_KS_IC
method: Dealiased_ETDRK4
noise_magnitude: 0.0
nx: 256
tmax: 150
xmax: 100.53096491487338
xmin: 0



In [86]:
ensemble_params = ConfigDict(KS_params_SALT)
print(ensemble_params)

Advection_basis_name: sin
E: 1
Forcing_basis_name: none
P: 1
S: 0
c_0: 0
c_1: 1
c_2: 1
c_3: 0.0
c_4: 1
dt: 0.25
equation_name: Kuramoto-Sivashinsky
initial_condition: Kassam_Trefethen_KS_IC
method: Dealiased_SETDRK4
noise_magnitude: 0.001
nx: 256
tmax: 150
xmax: 100.53096491487338
xmin: 0



Next, we specify a signal, by choosing a deterministic solver, 

In [87]:
signal_params.update(E=1,method='Dealiased_ETDRK4',nx = 256,P=0,S=0)
ensemble_params.update(E=128,sigma=0.01,P=32,stochastic_advection_basis='constant')


Now we continue to define a stochastic ensemble

Now we specify the models, by calling the class.

In [88]:
signal_model = ETD_KT_CM_JAX_Vectorised(signal_params)
ensemble_model = ETD_KT_CM_JAX_Vectorised(ensemble_params)

import numpy as np
initial_signal = initial_condition(signal_model.x, signal_params.E, signal_params.initial_condition)
initial_ensemble = initial_condition(ensemble_model.x, ensemble_params.E, ensemble_params.initial_condition)

available_resamplers = ", ".join(resamplers.keys())
print(available_resamplers)

multinomial, systematic, no_resampling, default


In [89]:
observation_locations = np.arange(10,signal_model.x.shape[0],25)
print(signal_model.x.shape)
print(observation_locations.shape)
pf_systematic = ParticleFilter(
    n_particles = ensemble_params.E,
    n_steps = 100,
    n_dim = initial_signal.shape[-1],
    forward_model = ensemble_model,
    signal_model = signal_model,
    sigma = 0.1,
    seed = 11,
    resampling="no_resampling",#'default',
    observation_locations = observation_locations,
)

(256,)
(10,)


In [90]:
da_steps = 10
final, all = pf_systematic.run(initial_ensemble, initial_signal, da_steps) #the final input is scan length? 

Put in the initial condition.

In [93]:
particles =jnp.concatenate([initial_ensemble[None,...], all[0]], axis=0)
signal = jnp.concatenate([initial_signal[None,...], all[1]], axis=0)
observations = all[2][:,:, observation_locations]
print(observations.shape)
print(particles.shape)
# time, n_ens, space

(10, 1, 10)
(11, 128, 256)


In [94]:
def plot(da_step):
    plt.plot(signal_model.x, signal[da_step,0,:], color='k',label='signal')
    plt.plot(signal_model.x, particles[da_step,:,:].T, color='b',label='particles',linewidth=0.1)
    if da_step > 0:
        plt.plot(signal_model.x[observation_locations], observations[da_step-1,0,:], 'ro',label='observations')
    plt.show()

interact(plot, da_step=(0, da_steps))

interactive(children=(IntSlider(value=5, description='da_step', max=10), Output()), _dom_classes=('widget-inte…

<function __main__.plot(da_step)>